# Scenario 2 subtask a: Forecasting with Vaccines

**Background:** 07/15/2021 in New York State. The Delta variant is beginning to spread. Vaccination distribution is still in its early phases (low rate, children <10 unable to be vaccinated). 

**TASK:** Forecast cases (I), hospitalizations (H), and deaths (D) over the next four weeks (28 days).

Specifications:
1. Set initial conditions from https://github.com/reichlab/covid19-forecast-hub/tree/master/data-truth
2. Incorporate data on vaccine uptake (https://covid.cdc.gov/covid-data-tracker/#vaccine-effectiveness)
3. Use the SEIRHD model from Scenario 1 now modified to include vaccination status.
4. Assume no masking or social distance mandates other than "unvaccinated people must wear masks"
5. Assume 50% of unvaccinated people actually comply
6. Assume all masks are surgical masks
7. $\beta = \beta_{uu}$, unvaccinated infecting unvaccinated
8. $\beta_{uv} = \beta_{vu} = (1-vaccine\_efficacy)*\beta_{uu}$
9. $\beta_{vv} = (1-vaccine\_efficacy)^2*{\beta_{uu}}^2$

In [20]:
import os
import urllib.request, json     
import pandas as pd
from IPython.display import HTML
from IPython import display
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
    load_petri_model,
    setup_petri_model,
    sample
)
import numpy as np
from typing import Iterable
from pyciemss.utils.interface_utils import (
    assign_interventions_to_timepoints,
    interventions_and_sampled_params_to_interval,
    convert_to_output_format
)
from pyciemss.utils import get_tspan
import matplotlib.pyplot as plt
import torch
from torch import tensor

from pyciemss.utils.toronto_ensemble_challenge_utils import get_case_hosp_death_data
import datetime

import sympy

### Load SEIRHD Model for this scenario

Utils to help update and change model for scenario

In [29]:
def update_AMR(SEIRD_model_url, SEIRD_model_path):
    with urllib.request.urlopen(SEIRD_model_url) as url:
        json_object = json.load(url)
        with open(SEIRD_model_path, "w") as outfile:
            json.dump(json_object, outfile)


def change_model_parameters(filename, new_params):
    # new params = [(param, value), (param, value)]
    with open(filename, 'r') as f:
        model = json.load(f)
        # Change initial parameters
        for (param, value) in new_params:
            for idx in model["semantics"]["ode"]["parameters"]:
                if idx["id"] == param:
                    idx["value"] = value
    return model

def change_model_initals(filename, new_initials):
    # new params = [(param, value), (param, value)]
    with open(filename, 'r') as f:
        model = json.load(f)
        # Change initial parameters
        for (initial, value) in new_initials:
            for idx in model["semantics"]["ode"]["initials"]:
                if idx["id"] == initial:
                    idx["value"] = value
    return model

def get_model_parameters(filename):
    with open(filename, 'r') as f:
        model = json.load(f)
        return [ idx for idx in model["semantics"]["ode"]["parameters"]]

def get_model_initial_conds(filename):
    with open(filename, 'r') as f:
        model = json.load(f)
        return [ idx for idx in model["semantics"]["ode"]["initials"]]



In [11]:
# S E I R H D
SEIRHD_model_path = "eval_scenario2_1_b.json"
SEIRHD_model_url = "https://raw.githubusercontent.com/indralab/mira/hackathon/notebooks/evaluation_2023.07/eval_scenario2_1_b.json"


# NOTE: This is only if you wish to update the AMR.


update_AMR(SEIRHD_model_url, SEIRHD_model_path)

In [4]:
# Simulation parametersa
num_samples = 1
start_time = 0
end_time = 28
num_timepoints = (end_time-start_time)*10 + 1
timepoints = get_tspan(start_time, end_time, num_timepoints)

### Determine which $\beta$ parameters correspond to different interactions between vaccinated/unvaccinated populations (where $\epsilon=$ vaccine efficacy)

#### Beta_UU: $\beta_{UU} = \beta$
- beta_0_0	to	beta_0_3


#### Beta_UV: $\beta_{UV} = (1-\epsilon)\beta$
- beta_3_0_0	to	beta_3_6_9

#### Beta_VU: $\beta_{VU} = (1-\epsilon)\beta$
- beta_1_0_0	to	beta_1_2_6

#### Beta_VV: $\beta_{VV} = (1-\epsilon)^2\beta$
- beta_3_0_0	to	beta_3_6_9

In [18]:
# Made up a numbers for hyperparameters

# vaccine efficacy values for different vaccines and doses
ve_JJ = 0.7
ve_M1 = 0.8
ve_M2 = 0.9
ve_P1 = 0.8
ve_P2 = 0.9

# Masking efficacy factors
I_mask = 0.25
S_mask = 0.5

# Unmasked infection rate initial guess
beta = 0.8



In [21]:
beta_vals_dict = {}

# UU parameters for this scenario model
beta_vals_dict['beta_0_3'] = beta
beta_vals_dict['beta_0_1'] = S_mask*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_0_2'] = I_mask*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_0_0'] = I_mask*S_mask*sympy.Symbol('beta_0_3')

# VU parameters for this scenario model
beta_vals_dict['beta_1_0_0'] = (1-ve_JJ)*S_mask*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_1_0_2'] = (1-ve_JJ)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_1_1_0_0'] = (1-ve_M1)*S_mask*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_1_1_0_2'] = (1-ve_M1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_1_1_1_0'] = (1-ve_M2)*S_mask*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_1_1_1_2'] = (1-ve_M2)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_1_2_0_0'] = (1-ve_P1)*S_mask*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_1_2_0_1'] = (1-ve_P1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_1_2_1_0'] = (1-ve_P2)*S_mask*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_1_2_1_2'] = (1-ve_P2)*sympy.Symbol('beta_0_3')

# UV parameters for this scenario model
beta_vals_dict['beta_3_0_0'] = (1-ve_JJ)*I_mask*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_3_0_1'] = (1-ve_JJ)*sympy.Symbol('beta_0_3')

beta_vals_dict['beta_3_3_0_0'] = (1-ve_M1)*I_mask*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_3_3_0_1'] = (1-ve_M1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_3_3_2_0'] = (1-ve_M2)*I_mask*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_3_3_2_1'] = (1-ve_M2)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_3_6_0_0'] = (1-ve_P1)*I_mask*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_3_6_0_1'] = (1-ve_P1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_3_6_2_0'] = (1-ve_P2)*I_mask*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_3_6_2_1'] = (1-ve_P2)*sympy.Symbol('beta_0_3')


# VV parameters for this scenario model
beta_vals_dict['beta_2_0'] = (1-ve_JJ)*(1-ve_JJ)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_1_0'] = (1-ve_M1)*(1-ve_JJ)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_1_1'] = (1-ve_M1)*(1-ve_JJ)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_2_0'] = (1-ve_P1)*(1-ve_JJ)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_2_1'] = (1-ve_P1)*(1-ve_JJ)*sympy.Symbol('beta_0_3')

beta_vals_dict['beta_2_3_0'] = (1-ve_M1)*(1-ve_M1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_3_1'] = (1-ve_M2)*(1-ve_M1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_3_2'] = (1-ve_M2)*(1-ve_M2)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_3_3'] = (1-ve_M1)*(1-ve_M2)*sympy.Symbol('beta_0_3')

beta_vals_dict['beta_2_4_0'] = (1-ve_JJ)*(1-ve_M1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_4_2'] = (1-ve_JJ)*(1-ve_M2)*sympy.Symbol('beta_0_3')

beta_vals_dict['beta_2_5_0'] = (1-ve_P1)*(1-ve_M1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_5_1'] = (1-ve_P2)*(1-ve_M1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_5_2'] = (1-ve_P2)*(1-ve_M2)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_5_3'] = (1-ve_P1)*(1-ve_M2)*sympy.Symbol('beta_0_3')

beta_vals_dict['beta_2_6_0'] = (1-ve_P1)*(1-ve_P1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_6_1'] = (1-ve_P2)*(1-ve_P1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_6_2'] = (1-ve_P2)*(1-ve_P2)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_6_3'] = (1-ve_P1)*(1-ve_P2)*sympy.Symbol('beta_0_3')

beta_vals_dict['beta_2_7_0'] = (1-ve_JJ)*(1-ve_P1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_7_2'] = (1-ve_JJ)*(1-ve_P2)*sympy.Symbol('beta_0_3')

beta_vals_dict['beta_2_8_0'] = (1-ve_M1)*(1-ve_P1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_8_1'] = (1-ve_M2)*(1-ve_P1)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_8_2'] = (1-ve_M2)*(1-ve_P2)*sympy.Symbol('beta_0_3')
beta_vals_dict['beta_2_8_3'] = (1-ve_M1)*(1-ve_P2)*sympy.Symbol('beta_0_3')

In [25]:
from mira.sources.askenet.petrinet import model_from_json_file
from mira.modeling.askenet.petrinet import AskeNetPetriNetModel
from mira.modeling import Model

# Convert AMR model to template_model
template_model = model_from_json_file(SEIRHD_model_path)

# Substitute rate laws using beta dictionary
for template in template_model.templates:
    for old, new in beta_vals_dict.items():
        template.rate_law = template.rate_law.subs(sympy.Symbol(old), new)

# Save template_model as AMR json file
AskeNetPetriNetModel(Model(template_model)).to_json_file(SEIRHD_model_path)


In [ ]:
# Change initial conditions to over-ride default stratification splits

In [27]:
# Test run the model with a 1-sample simulation
seirhd_samples = load_and_sample_petri_model(
        SEIRHD_model_path, num_samples, timepoints=timepoints)

In [28]:
seirhd_samples

{'data':      timepoint_id  sample_id     N_param  beta_0_3_param  beta_1_2_0_2_param  \
 0               0          0  19340000.0             0.8                 0.8   
 1               1          0  19340000.0             0.8                 0.8   
 2               2          0  19340000.0             0.8                 0.8   
 3               3          0  19340000.0             0.8                 0.8   
 4               4          0  19340000.0             0.8                 0.8   
 ..            ...        ...         ...             ...                 ...   
 276           276          0  19340000.0             0.8                 0.8   
 277           277          0  19340000.0             0.8                 0.8   
 278           278          0  19340000.0             0.8                 0.8   
 279           279          0  19340000.0             0.8                 0.8   
 280           280          0  19340000.0             0.8                 0.8   
 
      r_E_to_I_0_p

### Collect & parse data for NY State

In [32]:
# Use COVID-19 from Reich Lab for cases and deaths
infectious_period = 7
covid_data_df = get_case_hosp_death_data(US_region = "NY", infectious_period = infectious_period, make_csv=False)

/Users/mahs128/Repos/pyciemss/src/pyciemss/utils/toronto_ensemble_challenge_utils.py:25: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_cases = pd.read_csv(url)
/Users/mahs128/Repos/pyciemss/src/pyciemss/utils/toronto_ensemble_challenge_utils.py:37: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_deaths = pd.read_csv(url)
/Users/mahs128/Repos/pyciemss/src/pyciemss/utils/toronto_ensemble_challenge_utils.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regional_cases["case census"] = 0


In [40]:
from datetime import datetime, timedelta

print(covid_data_df.head())
print(" ")

start_day = '2021-07-15'

start_day_date = datetime(2021,7,15) - timedelta(28)
start_day_date


            case_census  hosp_census  cumulative_deaths
date                                                   
2020-04-10      72274.0            0               9579
2020-04-11      70557.0            0              10345
2020-04-17      53763.0            0              14371
2020-04-18      52107.0            1              14780
2020-04-24      40964.0            0              18132
 


datetime.datetime(2021, 6, 17, 0, 0)

In [ ]:

print(covid_data_df.loc[start_day])

df_idx = list(range(len(covid_data_df)))

print(covid_data_df.index)

start_day_idx = df_idx[str(covid_data_df.index) == start_day]
print(f"Start day index = {start_day_idx}")


In [ ]:

mapped_data = {}
mapped_data["timepoints"] = list(range(len(covid_data_df.index)))
mapped_data["I"] = covid_data_df["case_census"]
mapped_data["D"] = covid_data_df["cumulative_deaths"]
# write to CSV file
mapped_data_df = pd.DataFrame(mapped_data)
mapped_data_df.to_csv("mapped_data.csv", index=False)

In [6]:
SEIRHD_initials = get_model_initial_conds(SEIRHD_model_path)
print([(IC['target'], IC['expression']) for IC in SEIRHD_initials if IC['target'].startswith("S")])
sum_S_0 = sum([float(IC['expression']) for IC in SEIRHD_initials if IC['target'].startswith("S")])
print(f"Sum of susceptibles = {sum_S_0}")

[('S_unvaccinated_masked', '3640372.25000000'), ('S_unvaccinated_unmasked', '3640372.25000000'), ('S_vaccinated_j_and_j', '2426914.8333333335'), ('S_vaccinated_moderna_1dose', '1213457.4166666667'), ('S_vaccinated_moderna_2dose', '1213457.4166666667'), ('S_vaccinated_pfizer_1dose', '1213457.4166666667'), ('S_vaccinated_pfizer_2dose', '1213457.4166666667')]
Sum of susceptibles = 14561488.999999998


In [7]:
# Test run the model with a 1-sample simulation
seirhd_samples = load_and_sample_petri_model(
        SEIRHD_model_path, num_samples, timepoints=timepoints)

In [8]:
#[(seirhd_samples['data'].columns[i],v) for v,i in enumerate(seirhd_samples['data'].iloc[0])]
IC = zip([c for c in seirhd_samples['data'].columns],[v for v in seirhd_samples['data'].iloc[0]])
#print([cv for cv in IC if cv[0].startswith("S")])
(sum([cv[1] for cv in IC if cv[0].startswith("S")]) - sum_S_0)


-0.24999999813735485

In [47]:
SEIRHD_parameters = get_model_parameters(SEIRHD_model_path)
SEIRHD_param_names = [p['id'] for p in SEIRHD_parameters]

In [9]:
SEIRHD_model = load_petri_model(SEIRHD_model_path)
SEIRHD_model


/Users/mahs128/Repos/pyciemss/src/pyciemss/PetriNetODE/base.py:342: UserWarning: Parameter (('S_unvaccinated_masked', ('identity', 'ido:0000514'), ('masking', 'masked'), ('vaccinated', 'unvaccinated')), ('E_unvaccinated_masked', ('identity', 'apollosv:0000154'), ('masking', 'masked'), ('vaccinated', 'unvaccinated')), ('I_unvaccinated_masked', ('identity', 'ido:0000511'), ('masking', 'masked'), ('vaccinated', 'unvaccinated')), 'ControlledConversion', 'rate') has value None and will be set to Uniform(0, 1). This is likely to be an error.
  warnings.warn(warnings_string)
/Users/mahs128/Repos/pyciemss/src/pyciemss/PetriNetODE/base.py:342: UserWarning: Parameter (('S_unvaccinated_masked', ('identity', 'ido:0000514'), ('masking', 'masked'), ('vaccinated', 'unvaccinated')), ('E_unvaccinated_masked', ('identity', 'apollosv:0000154'), ('masking', 'masked'), ('vaccinated', 'unvaccinated')), ('I_unvaccinated_unmasked', ('identity', 'ido:0000511'), ('masking', 'unmasked'), ('vaccinated', 'unvaccin

ScaledNormalNoisePetriNetODESystem(
	N = Uniform(low: 17406000.0, high: 21274000.0),
	beta_0_0 = Uniform(low: 0.7200000286102295, high: 0.8799999952316284),
	(('S_unvaccinated_masked', ('identity', 'ido:0000514'), ('masking', 'masked'), ('vaccinated', 'unvaccinated')), ('E_unvaccinated_masked', ('identity', 'apollosv:0000154'), ('masking', 'masked'), ('vaccinated', 'unvaccinated')), ('I_unvaccinated_masked', ('identity', 'ido:0000511'), ('masking', 'masked'), ('vaccinated', 'unvaccinated')), 'ControlledConversion', 'rate') = Uniform(low: 0.0, high: 1.0),
	beta_0_1 = Uniform(low: 0.7200000286102295, high: 0.8799999952316284),
	(('S_unvaccinated_masked', ('identity', 'ido:0000514'), ('masking', 'masked'), ('vaccinated', 'unvaccinated')), ('E_unvaccinated_masked', ('identity', 'apollosv:0000154'), ('masking', 'masked'), ('vaccinated', 'unvaccinated')), ('I_unvaccinated_unmasked', ('identity', 'ido:0000511'), ('masking', 'unmasked'), ('vaccinated', 'unvaccinated')), 'ControlledConversion',

In [ ]:
# Set initial conditions for strata
# As of June 24, 2021, general masking and social distancing are no longer in place in NY state, BUT all unvaccinated people need to wear masks
# Assume 50% of unvaccinated people comply and wear masks
S_0_9_masked
S_0_9_unmasked
I_0_9_masked
I_0_9_unmasked

S_10_unvaccinated_masked
S_10_unvaccinated_unmasked
I_10_unvaccinated_masked
I_10_unvaccinated_unmasked